In [1]:
import pulp

In [2]:
import numpy as np
import pandas as pd

In [76]:
members = range(1,31)
days = range(1,29)
roles = range(1,18)

In [77]:
# defining the variables
x = pulp.LpVariable.dicts("x",(members,days,roles),0,1,pulp.LpInteger)

In [78]:
y = pulp.LpVariable.dicts("y",(members),0,None,pulp.LpInteger)

In [79]:
# binary variable
b = pulp.LpVariable.dicts("b",(members),0,1,pulp.LpInteger)

In [80]:
# objective
model = pulp.LpProblem("Toastmasters", pulp.LpMaximize)
model += pulp.lpSum(y[si] for si in members)

In [81]:
# 17 roles per meeting
for d in days:
    model += pulp.lpSum(x[m][d][r] for m in members for r in roles) == 17

In [82]:
# Constraint not satisfied depending on the number of members
# every member has at least done a role once
# for m in members:
#     for r in roles:
#         model += pulp.lpSum(x[m][d][r] for d in days) >= 1

In [83]:
# only one role per member per meeting
for m in members:
    for d in days:
        model += pulp.lpSum(x[m][d][r] for r in roles) <= 1

In [84]:
# every role in the meeting must be occupied
for d in days:
    for r in roles:
        model += pulp.lpSum(x[m][d][r] for m in members) == 1

### Adding soft constraints

In [85]:
# number of roles that a member has done
        
for m in members:
    model += pulp.lpSum(x[m][d][r] for d in days for r in roles) - y[m] == 0

In [86]:
# using big M method to get abs(y) >= 15

M = 1e4

for m in members:
    model += y[m] + (M * b[m]) >= 15
    model += -y[m] + (M * (1 - b[m])) >= 15

## Solving the Model

In [87]:
# Solve ILP problem and post-processing to get the summary
model.solve()

1

In [88]:
print("Status:", pulp.LpStatus[model.status])

Status: Optimal


In [19]:
# model

In [89]:
for vi in model.variables():
    if vi.varValue == 1:
        print(f"{vi.name} = {vi.varValue:.0f}")

x_10_11_12 = 1
x_10_1_8 = 1
x_10_20_15 = 1
x_10_21_4 = 1
x_10_22_5 = 1
x_10_23_12 = 1
x_10_25_12 = 1
x_10_26_9 = 1
x_10_28_5 = 1
x_10_2_15 = 1
x_10_3_12 = 1
x_10_4_3 = 1
x_10_5_16 = 1
x_10_6_5 = 1
x_10_7_10 = 1
x_10_8_17 = 1
x_10_9_9 = 1
x_11_10_6 = 1
x_11_12_14 = 1
x_11_21_9 = 1
x_11_22_16 = 1
x_11_23_5 = 1
x_11_25_7 = 1
x_11_26_8 = 1
x_11_27_9 = 1
x_11_28_16 = 1
x_11_2_16 = 1
x_11_3_7 = 1
x_11_5_9 = 1
x_11_6_14 = 1
x_11_7_11 = 1
x_11_8_2 = 1
x_11_9_5 = 1
x_12_12_16 = 1
x_12_17_13 = 1
x_12_1_6 = 1
x_12_20_7 = 1
x_12_21_7 = 1
x_12_22_2 = 1
x_12_24_5 = 1
x_12_26_7 = 1
x_12_28_2 = 1
x_12_3_5 = 1
x_12_4_17 = 1
x_12_5_11 = 1
x_12_6_8 = 1
x_12_7_14 = 1
x_12_8_15 = 1
x_12_9_16 = 1
x_13_15_13 = 1
x_13_18_12 = 1
x_13_19_3 = 1
x_13_20_5 = 1
x_13_22_11 = 1
x_13_24_4 = 1
x_13_26_11 = 1
x_13_27_15 = 1
x_13_28_10 = 1
x_13_3_9 = 1
x_13_4_4 = 1
x_13_6_10 = 1
x_13_7_8 = 1
x_13_8_3 = 1
x_13_9_3 = 1
x_14_17_4 = 1
x_14_20_17 = 1
x_14_21_6 = 1
x_14_22_7 = 1
x_14_24_1 = 1
x_14_25_14 = 1
x_14_26_3 = 1
x_14_

## Converting to a dataframe for a clean output

In [90]:
myperson = []
myday = []
myrole = []
value = []

for vi in model.variables():
    
    if vi.name.split("_")[0] == 'x':
    
        person = vi.name.split("_")[1]
        day = vi.name.split("_")[2]
        role = vi.name.split("_")[3]

        myperson.append(person)
        myday.append(day)
        myrole.append(role)
        value.append(vi.varValue)
    
data = {'Person':myperson, 'Day':myday, 'Role':myrole, 'Value':value} 

df = pd.DataFrame(data)

In [92]:
# df.to_csv('role_optimazation.csv', index = False)

In [91]:
df

,Person,Day,Role,Value
0,10,10,1,0.0
1,10,10,10,0.0
2,10,10,11,0.0
3,10,10,12,0.0
4,10,10,13,0.0
...,...,...,...,...
14275,9,9,5,0.0
14276,9,9,6,0.0
14277,9,9,7,0.0
14278,9,9,8,0.0


In [92]:
roles = [
'Ah Counter', 
'Grammarian',
'Timer', 
'Toastmaster', 
'Meeting speaker 1',
'Meeting speaker 2',
'Meeting speaker 3',
'meeting speaker 4',
'Speech Evaluator 1',
'Speech Evaluator 2',
'Speech Evaluator 3',
'Speech Evaluator 4',
'General evaluator', 
'Table topic master', 
'Table topic evalauator 1',
'Table topic evalauator 2',
'Harkmaster' ]

person = [
    'Member 1',
'Member 2',
'Member 3',
'Member 4',
'Member 5',
'Member 6',
'Member 7',
'Member 8',
'Member 9',
'Member 10',
'Member 11',
'Member 12',
'Member 13',
'Member 14',
'Member 15',
'Member 16',
'Member 17',
'Member 18',
'Member 19',
'Member 20',
'Member 21',
'Member 22',
'Member 23',
'Member 24',
'Member 25',
'Member 26',
'Member 27',
'Member 28',
'Member 29',
'Member 30',
]

In [93]:
person_dict = {}
for i in enumerate(person):
    person_dict[i[0] + 1] = i[1]
    
role_dict = {}
for i in enumerate(roles):
    role_dict[i[0] + 1] = i[1]

In [94]:
df['Name'] = df['Person'].apply(lambda x:person_dict[int(x)])
df['Assignment'] = df['Role'].apply(lambda x:role_dict[int(x)])

In [95]:
df

,Person,Day,Role,Value,Name,Assignment
0,10,10,1,0.0,Member 10,Ah Counter
1,10,10,10,0.0,Member 10,Speech Evaluator 2
2,10,10,11,0.0,Member 10,Speech Evaluator 3
3,10,10,12,0.0,Member 10,Speech Evaluator 4
4,10,10,13,0.0,Member 10,General evaluator
...,...,...,...,...,...,...
14275,9,9,5,0.0,Member 9,Meeting speaker 1
14276,9,9,6,0.0,Member 9,Meeting speaker 2
14277,9,9,7,0.0,Member 9,Meeting speaker 3
14278,9,9,8,0.0,Member 9,meeting speaker 4


In [96]:
def mapper(row):
    if row['Value'] == 1:
        return str(int(row['Role']))
    return str(int(row['Value']))

In [97]:
df['y'] = df.apply(mapper,axis = 1)

In [98]:
df

,Person,Day,Role,Value,Name,Assignment,y
0,10,10,1,0.0,Member 10,Ah Counter,0
1,10,10,10,0.0,Member 10,Speech Evaluator 2,0
2,10,10,11,0.0,Member 10,Speech Evaluator 3,0
3,10,10,12,0.0,Member 10,Speech Evaluator 4,0
4,10,10,13,0.0,Member 10,General evaluator,0
...,...,...,...,...,...,...,...
14275,9,9,5,0.0,Member 9,Meeting speaker 1,0
14276,9,9,6,0.0,Member 9,Meeting speaker 2,0
14277,9,9,7,0.0,Member 9,Meeting speaker 3,0
14278,9,9,8,0.0,Member 9,meeting speaker 4,0


In [104]:
df['z'] = df['y'].apply(int)

In [105]:
table = pd.pivot_table(df, values='z', index=['Name'],
                    columns=['Day'], aggfunc=np.sum)

table

Day,1,10,11,12,13,14,15,16,17,18,...,26,27,28,3,4,5,6,7,8,9
Name,,,,,,,,,,,,,,,,,,,,,
Member 1,12,0,0,0,0,8,0,0,7,0,...,14,2,12,0,15,15,0,12,0,2
Member 10,8,0,12,0,0,0,0,0,0,0,...,9,0,5,12,3,16,5,10,17,9
Member 11,0,6,0,14,0,0,0,0,0,0,...,8,9,16,7,0,9,14,11,2,5
Member 12,6,0,0,16,0,0,0,0,13,0,...,7,0,2,5,17,11,8,14,15,16
Member 13,0,0,0,0,0,0,13,0,0,12,...,11,15,10,9,4,0,10,8,3,3
Member 14,0,0,0,0,0,0,0,0,4,0,...,3,3,6,0,11,12,15,5,4,4
Member 15,3,0,0,0,0,0,0,6,0,0,...,16,12,14,6,8,4,9,16,9,0
Member 16,0,0,0,0,17,16,0,9,0,4,...,0,8,4,4,0,6,2,9,0,17
Member 17,1,0,0,0,0,9,0,0,0,8,...,0,16,0,11,10,2,4,0,7,6


In [107]:
df.groupby('Name').sum()['Value']

Name
Member 1     15.0
Member 10    17.0
Member 11    16.0
Member 12    16.0
Member 13    15.0
Member 14    15.0
Member 15    15.0
Member 16    15.0
Member 17    15.0
Member 18    15.0
Member 19    15.0
Member 2     15.0
Member 20    15.0
Member 21    15.0
Member 22    18.0
Member 23    18.0
Member 24    17.0
Member 25    17.0
Member 26    16.0
Member 27    17.0
Member 28    15.0
Member 29    16.0
Member 3     16.0
Member 30    15.0
Member 4     16.0
Member 5     17.0
Member 6     17.0
Member 7     15.0
Member 8     15.0
Member 9     17.0
Name: Value, dtype: float64

In [108]:
# df.to_csv('role_optimization_final_2.csv', index = False)